In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

In [6]:
# Only keep the dates we want
comments = comments.query('updatedAt > @start_date and updatedAt < @end_date')
issues = issues.query('updatedAt > @start_date and updatedAt < @end_date')
prs = prs.query('updatedAt > @start_date and updatedAt < @end_date')

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/nteract](https://github.com/nteract)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-03** to **2019-08-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 90 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                  org                           repo  author
  87           nteract                      bookstore      43
  89           nteract                       hydrogen      40
  93           nteract                        nteract      32
  95           nteract                      papermill      20
  96           nteract                      scrapbook       5
  88           nteract                       commuter       4
  97           nteract                       semiotic       4
  91           nteract                meeting-minutes       3
  92           nteract                 minimal-plotly       2
  99           nteract                           vdom       2
  100          nteract                  vega-embed-v3       2
  90           nteract                   kernel-relay       1
  94           nteract                     nteract.io       1
  98           nteract                  semiotic-docs       1
  72        jupyterlab                     jupyterlab    1079
  76        jupyterlab                 jupyterlab-git      65
  74        jupyterlab       jupyterlab-data-explorer      60
  83        jupyterlab              jupyterlab_server      45
  75        jupyterlab                jupyterlab-demo      40
  71        jupyterlab              jupyter-renderers      35
  73        jupyterlab          jupyterlab-commenting      25
  77        jupyterlab              jupyterlab-github      25
  68        jupyterlab                       debugger      17
  70        jupyterlab      extension-cookiecutter-ts      15
  86        jupyterlab             theme-cookiecutter      15
  69        jupyterlab      extension-cookiecutter-js      10
  81        jupyterlab                 jupyterlab-toc      10
  85        jupyterlab  scipy2019-jupyterlab-tutorial      10
  78        jupyterlab               jupyterlab-latex       5
  79        jupyterlab    jupyterlab-metadata-service       5
  ..               ...                            ...     ...
  11           jupyter             enterprise_gateway      33
  23           jupyter                      nbconvert      31
  14           jupyter                   jupyter-book      28
  10           jupyter                  docker-stacks      24
  18           jupyter                 jupyter_client      17
  32           jupyter                      telemetry      16
  16           jupyter                 jupyter-sphinx      12
  29           jupyter                      qtconsole      12
  19           jupyter                   jupyter_core       9
  20           jupyter                 jupyter_server       9
  17           jupyter              jupyter.github.io       8
  13           jupyter                        jupyter       5
  22           jupyter                          nb2kg       5
  26           jupyter                       nbviewer       5
  15           jupyter              jupyter-packaging       3
  21           jupyter                 kernel_gateway       3
  24           jupyter                         nbdime       3
  9            jupyter                         design       2
  12           jupyter                     governance       1
  28           jupyter                   ops-handbook       1
  31           jupyter                        roadmap       1
  6            ipython                        ipython     156
  4            ipython                      ipykernel      21
  8            ipython                      traitlets      15
  5            ipython                    ipyparallel      12
  7            ipython                ipython-website       3
  0    binder-examples                     data-quilt       2
  1    binder-examples             getting-data-xroot       2
  2    binder-examples                          latex       2
  3    binder-examples               python-conda_pip       2
  
  [101 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
   

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:           username  authored authorAssociation
    0        fomightez         2       CONTRIBUTOR
    1      matheusmota         2              NONE
    2  matthewfeickert         2       CONTRIBUTOR
    3        yuvipanda         2            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 90 days',
    width: 1000
  }), Chart({
    data:                  username  authored authorAssociation
    0                 Carreau        75            MEMBER
    26                  minrk        12            MEMBER
    2                   Jamim         9       CONTRIBUTOR
    20                hroncok         9       CONTRIBUTOR
    4          LucianaMarques         6       CONTRIBUTOR
    6              OriolAbril         6       CONTRIBUTOR
    8                    alok         6       CONTRIBUTOR
    27             pierstitus         3       CONTRIBUTOR
    22                  k-zen         3       CONTRIBUTOR
    23            kevin-bates         3       CONTRIBUTOR
    24                 kmaork         3       CONTRIBUTOR
    25                minho42         3       CONTRIBUTOR
    30               stonebig         3       CONTRIBUTOR
    28                  rgson         3       CONTRIBUTOR
    29               steff456         3              NONE
    31                sunpoet         3       CONTRIBUTOR
    32             sysadmin75         3              NONE
    33              tacaswell         3       CONTRIBUTOR
    21             juanis2112         3       CONTRIBUTOR
    17          frenzymadness         3       CONTRIBUTOR
    19               gpotter2         3       CONTRIBUTOR
    18             gooverdian         3       CONTRIBUTOR
    1                 Chilipp         3       CONTRIBUTOR
    16               eobrie16         3       CONTRIBUTOR
    15                 ellert         3       CONTRIBUTOR
    14           danielballan         3       CONTRIBUTOR
    13                 daharn         3       CONTRIBUTOR
    12         brandonwillard         3       CONTRIBUTOR
    11                blueyed         3       CONTRIBUTOR
    10               bashtage         3       CONTRIBUTOR
    9                 anntzer         3       CONTRIBUTOR
    7               Pragnya02         3       CONTRIBUTOR
    5                   MSeal         3       CONTRIBUTOR
    3   LittleLightLittleFire         3       CONTRIBUTOR
    34                zzzz-qq         3       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 90 days',
    width: 1000
  }), Chart({
    data:           username  authored authorAssociation

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org                repo   username  closed
    0  binder-examples          data-quilt    betatim       2
    1  binder-examples  getting-data-xroot  choldgraf       2
    2  binder-examples               latex  choldgraf       2
    3  binder-examples    python-conda_pip  choldgraf       2,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 90 days',
    width: 1000
  }), Chart({
    data:         org             repo   username  closed
    4   ipython        ipykernel    Carreau       3
    5   ipython        ipykernel  blink1073       3
    6   ipython        ipykernel      minrk      12
    7   ipython        ipykernel   willingc       3
    8   ipython      ipyparallel      minrk      12
    9   ipython          ipython    Carreau     156
    10  ipython  ipython-website  blink1073       3
    11  ipython        traitlets   rmorshea      15,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 90 days',
    width: 1000
  }), Chart({
    data:         org                repo       username  closed
    12  jupyter              design        Carreau       1
    13  jupyter              design      blink1073       1
    14  jupyter       docker-stacks          minrk       6
    15  jupyter       docker-stacks        parente      18
    16  jupyter  enterprise_gateway      akchinSTC       3
    17  jupyter  enterprise_gateway    kevin-bates      23
    18  jupyter  enterprise_gateway       lresende       7
    19  jupyter          governance      choldgraf       1
    20  jupyter             jupyter      choldgraf       3
    21  jupyter             jupyter        parente       2
    22  jupyter        jupyter-book      choldgraf      28
    23  jupyter   jupyter-packaging      blink1073       2
    24  jupyter   jupyter-packaging        vidartf       1
    25  jupyter      jupyter-sphinx  SylvainCorlay       1
    26  jupyter      jupyter-sphinx       akhmerov       9
    27  jupyter      jupyter-sphinx       jbweston       2
    28  jupyter   jupyter.github.io        Carreau       1
    29  jupyter   jupyter.github.io           Ruv7       1
    30  jupyter   jupyter.github.io         afshin       1
    31  jupyter   jupyter.github.io      choldgraf       2
    32  jupyter   jupyter.github.io      ellisonbg       1
    33  jupyter   jupyter.github.io     jasongrout       2
    34  jupyter      jupyter_client          MSeal       2
    35  jupyter      jupyter_client  SylvainCorlay       5
    36  jupyter      jupyter_client      blink1073       1
    37  jupyter      jupyter_client          minrk       4
    38  jupyter      jupyter_client         mpacer       1
    39  jupyter      jupyter_client         rgbkrk       4
    40  jupyter        jupyter_core          minrk       1
    41  jupyter        jupyter_core         rgbkrk       1
    ..      ...                 ...            ...     ...
    43  jupyter      jupyter_server  SylvainCorlay       4
    44  jupyter      jupyter_server      blink1073       1
    45  jupyter      jupyter_server         ivanov       1
    46  jupyter      jupyter_server       rolweber       3
    47  jupyter      kernel_gateway       rolweber       3
    48  jupyter               nb2kg    kevin-bates       4
    49  jupyter               nb2kg       lresende       1
    50  jupyter           nbconvert          MSeal      26
    51  jupyter           nbconvert       t-makaro       4
    52  jupyter           nbconvert       willingc       1
    53  jupyter              nbdime        vidartf       3
    54  jupyter            nbgrader       

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                           repo  number  \
  0   binder-examples                   jupyter-rise       2   
  1           ipython                      ipykernel      27   
  2           ipython                    ipyparallel      24   
  3           ipython                        ipython     128   
  4           ipython               ipython-in-depth       3   
  5           jupyter                  accessibility       1   
  6           jupyter                     dashboards       1   
  7           jupyter                         design       1   
  8           jupyter                  docker-stacks      16   
  9           jupyter             enterprise_gateway       8   
  10          jupyter                        jupyter       5   
  11          jupyter                   jupyter-book      22   
  12          jupyter                  jupyter-drive       1   
  13          jupyter                 jupyter-sphinx       9   
  14          jupyter              jupyter.github.io       1   
  15          jupyter                 jupyter_client       5   
  16          jupyter                   jupyter_core       2   
  17          jupyter            jupyter_kernel_test       1   
  18          jupyter                 jupyter_server      12   
  19          jupyter                 kernel_gateway       1   
  20          jupyter                          nb2kg       2   
  21          jupyter                      nbconvert      17   
  22          jupyter                         nbdime       5   
  23          jupyter                       nbgrader      28   
  24          jupyter                       nbviewer       6   
  25          jupyter                       notebook      91   
  26          jupyter                      qtconsole       4   
  27          jupyter                    repo2docker      29   
  28          jupyter                       steering       1   
  29          jupyter                      telemetry       1   
  ..              ...                            ...     ...   
  69       jupyterlab      extension-cookiecutter-ts       5   
  70       jupyterlab              jupyter-renderers      15   
  71       jupyterlab                     jupyterlab     958   
  72       jupyterlab            jupyterlab-celltags      25   
  73       jupyterlab          jupyterlab-commenting      55   
  74       jupyterlab       jupyterlab-data-explorer      90   
  75       jupyterlab                 jupyterlab-git      53   
  76       jupyterlab              jupyterlab-github       5   
  77       jupyterlab        jupyterlab-google-drive       5   
  78       jupyterlab    jupyterlab-metadata-service       5   
  79       jupyterlab              jupyterlab-monaco       5   
  80       jupyterlab           jupyterlab-telemetry      10   
  81       jupyterlab                 jupyterlab-toc      10   
  82       jupyterlab                jupyterlab_apod       5   
  83       jupyterlab                 jupyterlab_app       5   
  84       jupyterlab  scipy2018-jupyterlab-tutorial       5   
  85       jupyterlab                   team-compass      24   
  86          nteract                      bookstore      15   
  87          nteract                    coffee_boat       1   
  88          nteract                       commuter       2   
  89          nteract             commuter-on-glitch       1   
  90          nteract                             dx       3   
  91          nteract                       hydrogen      12   
  92          nteract                        nteract      29   
  93          nteract                     nteract.io       2   
  94          nteract                   oauth-server       1   
  95          nteract                      papermill      22   
  96          nteract                      scrapbook       3   
  97          nteract                       semiotic       6   
  98          nteract                           vdom       1   
  
               

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                      bokeh       2   
  1   binder-examples                 demo-julia       2   
  2           ipython                  ipykernel       9   
  3           ipython                ipyparallel       3   
  4           ipython                    ipython     132   
  5           ipython                  traitlets       6   
  6           jupyter              atom-notebook       1   
  7           jupyter              docker-stacks      17   
  8           jupyter         enterprise_gateway      30   
  9           jupyter                    jupyter       5   
  10          jupyter               jupyter-book      28   
  11          jupyter             jupyter-sphinx      15   
  12          jupyter          jupyter.github.io       1   
  13          jupyter             jupyter_client       8   
  14          jupyter               jupyter_core      10   
  15          jupyter             jupyter_server       9   
  16          jupyter             kernel_gateway       3   
  17          jupyter                      nb2kg       4   
  18          jupyter                  nbconvert      30   
  19          jupyter                     nbdime       4   
  20          jupyter                   nbgrader      47   
  21          jupyter                   nbviewer       7   
  22          jupyter                   notebook      63   
  23          jupyter                  qtconsole       8   
  24          jupyter                repo2docker      24   
  25          jupyter                  telemetry       3   
  26          jupyter                  terminado       1   
  27  jupyter-widgets                 ipyleaflet      84   
  28  jupyter-widgets                 ipywidgets     164   
  29  jupyter-widgets         jupyterlab-sidecar       4   
  ..              ...                        ...     ...   
  46       jupyterhub                nbgitpuller      21   
  47       jupyterhub             oauthenticator       2   
  48       jupyterhub             systemdspawner       2   
  49       jupyterhub               team-compass      49   
  50       jupyterhub    the-littlest-jupyterhub     100   
  51       jupyterhub           tmpauthenticator       2   
  52       jupyterhub              traefik-proxy      10   
  53       jupyterhub                yarnspawner       2   
  54       jupyterhub     zero-to-jupyterhub-k8s      37   
  55       jupyterlab  extension-cookiecutter-js      10   
  56       jupyterlab          jupyter-renderers      40   
  57       jupyterlab                 jupyterlab    1220   
  58       jupyterlab      jupyterlab-commenting       5   
  59       jupyterlab   jupyterlab-data-explorer      70   
  60       jupyterlab            jupyterlab-demo      10   
  61       jupyterlab             jupyterlab-git      67   
  62       jupyterlab          jupyterlab-github       5   
  63       jupyterlab             jupyterlab-toc      10   
  64       jupyterlab          jupyterlab_server       6   
  65       jupyterlab               team-compass       1   
  66          nteract                  bookstore      16   
  67          nteract                   commuter       2   
  68          nteract         commuter-on-glitch       1   
  69          nteract                   hydrogen     118   
  70          nteract             minimal-plotly       2   
  71          nteract                    nteract      56   
  72          nteract                  papermill      18   
  73          nteract                  scrapbook       1   
  74          nteract                   semiotic      17   
  75          nteract                       vdom       3   
  
                                  org/repo  
  0                  binder-examples/bokeh  
  1             binder-examples/demo-julia  
  2                      ipython/ipykernel  
  3                    ipython/ipyparallel  
  4                        ipython/ipy

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             org                       repo     kind   count
    70   jupyterlab          jupyter-renderers  created    15.0
    71   jupyterlab                 jupyterlab  created   958.0
    72   jupyterlab        jupyterlab-celltags  created    25.0
    73   jupyterlab      jupyterlab-commenting  created    55.0
    74   jupyterlab   jupyterlab-data-explorer  created    90.0
    75   jupyterlab             jupyterlab-git  created    53.0
    80   jupyterlab       jupyterlab-telemetry  created    10.0
    81   jupyterlab             jupyterlab-toc  created    10.0
    85   jupyterlab               team-compass  created    24.0
    106  jupyterlab  extension-cookiecutter-js  created     0.0
    180  jupyterlab          jupyter-renderers   closed    40.0
    181  jupyterlab                 jupyterlab   closed  1220.0
    182  jupyterlab        jupyterlab-celltags   closed     0.0
    183  jupyterlab      jupyterlab-commenting   closed     5.0
    184  jupyterlab   jupyterlab-data-explorer   closed    70.0
    185  jupyterlab             jupyterlab-git   closed    67.0
    190  jupyterlab       jupyterlab-telemetry   closed     0.0
    191  jupyterlab             jupyterlab-toc   closed    10.0
    195  jupyterlab               team-compass   closed     1.0
    216  jupyterlab  extension-cookiecutter-js   closed    10.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 90 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                  org                repo     kind  count
    32   jupyter-widgets          ipyleaflet  created   56.0
    33   jupyter-widgets          ipywidgets  created  198.0
    34   jupyter-widgets  jupyterlab-sidecar  created   13.0
    35   jupyter-widgets           pythreejs  created   36.0
    36   jupyter-widgets            tutorial  created    8.0
    103  jupyter-widgets        midicontrols  created    0.0
    142  jupyter-widgets          ipyleaflet   closed   84.0
    143  jupyter-widgets          ipywidgets   closed  164.0
    144  jupyter-widgets  jupyterlab-sidecar   closed    4.0
    145  jupyter-widgets           pythreejs   closed   20.0
    146  jupyter-widgets            tutorial   closed   12.0
    213  jupyter-widgets        midicontrols   closed    4.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 90 days for jupyter-widgets',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:          org              repo     kind  count
    1    ipython         ipykernel  created   27.0
    2    ipython       ipyparallel  created   24.0
    3    ipython           ipython  created  128.0
    4    ipython  ipython-in-depth  created    3.0
    101  ipython         traitlets  created    0.0
    111  ipython         ipykernel   closed    9.0
    112 

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                       repo   time_open  \
  0   binder-examples                      bokeh    2.983160   
  1   binder-examples                 demo-julia    6.085058   
  2           ipython                  ipykernel   87.525336   
  3           ipython                ipyparallel   62.380706   
  4           ipython                    ipython    6.123883   
  5           ipython                  traitlets   53.081319   
  6           jupyter              atom-notebook   73.856030   
  7           jupyter              docker-stacks   11.228750   
  8           jupyter         enterprise_gateway    6.643426   
  9           jupyter                    jupyter   21.940382   
  10          jupyter               jupyter-book   28.275017   
  11          jupyter             jupyter-sphinx    4.944387   
  12          jupyter          jupyter.github.io    2.754988   
  13          jupyter             jupyter_client   10.278553   
  14          jupyter               jupyter_core  324.044091   
  15          jupyter             jupyter_server  164.074618   
  16          jupyter             kernel_gateway  346.509884   
  17          jupyter                      nb2kg   16.084855   
  18          jupyter                  nbconvert  438.766684   
  19          jupyter                     nbdime   16.385758   
  20          jupyter                   nbgrader  170.229317   
  21          jupyter                   nbviewer   25.312731   
  22          jupyter                   notebook  278.093495   
  23          jupyter                  qtconsole    6.688837   
  24          jupyter                repo2docker   22.855793   
  25          jupyter                  telemetry    0.930949   
  26          jupyter                  terminado    0.076042   
  27  jupyter-widgets                 ipyleaflet    9.527373   
  28  jupyter-widgets                 ipywidgets    7.529421   
  29  jupyter-widgets         jupyterlab-sidecar   16.910301   
  ..              ...                        ...         ...   
  46       jupyterhub                nbgitpuller  272.757106   
  47       jupyterhub             oauthenticator  109.493565   
  48       jupyterhub             systemdspawner   20.067164   
  49       jupyterhub               team-compass   31.010162   
  50       jupyterhub    the-littlest-jupyterhub   86.692396   
  51       jupyterhub           tmpauthenticator   50.088738   
  52       jupyterhub              traefik-proxy   30.717477   
  53       jupyterhub                yarnspawner    1.391169   
  54       jupyterhub     zero-to-jupyterhub-k8s    3.264178   
  55       jupyterlab  extension-cookiecutter-js    0.964578   
  56       jupyterlab          jupyter-renderers  273.376128   
  57       jupyterlab                 jupyterlab    4.883113   
  58       jupyterlab      jupyterlab-commenting   14.237836   
  59       jupyterlab   jupyterlab-data-explorer   41.514595   
  60       jupyterlab            jupyterlab-demo   29.906973   
  61       jupyterlab             jupyterlab-git    8.193553   
  62       jupyterlab          jupyterlab-github    0.072361   
  63       jupyterlab             jupyterlab-toc    1.095272   
  64       jupyterlab          jupyterlab_server    8.985532   
  65       jupyterlab               team-compass    7.733113   
  66          nteract                  bookstore   13.027894   
  67          nteract                   commuter   72.565521   
  68          nteract         commuter-on-glitch  188.914167   
  69          nteract                   hydrogen  192.071082   
  70          nteract             minimal-plotly   23.436053   
  71          nteract                    nteract  115.842876   
  72          nteract                  papermill   20.898322   
  73          nteract                  scrapbook   36.124907   
  74          nteract                   semiotic   21.760567   
  75          nteract                       vdom  108.073310   
  
               

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 90 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                  author  count authorAssociation
    4             choldgraf     16            MEMBER
    11             stwunsch     16              NONE
    2               betatim     10            MEMBER
    10      matthewfeickert      6              NONE
    1   3keepmovingforward3      4              NONE
    6           cornhundred      4              NONE
    8   mathematicalmichael      4              NONE
    12         taylorreiter      4              NONE
    0                0lorak      2              NONE
    3              cboettig      2            MEMBER
    5             chrisburr      2              NONE
    7          davidanthoff      2       CONTRIBUTOR
    9           matheusmota      2              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 90 days',
    width: 1000
  }), Chart({
    data:                      author  count authorAssociation
    7                   Carreau    445            MEMBER
    151                rmorshea     57      COLLABORATOR
    132                   minrk     51            MEMBER
    15           LucianaMarques     48       CONTRIBUTOR
    91                 gpotter2     39       CONTRIBUTOR
    115             kevin-bates     36       CONTRIBUTOR
    164               tacaswell     30       CONTRIBUTOR
    60               ccordoba12     27       CONTRIBUTOR
    95                  hroncok     24       CONTRIBUTOR
    18                    MSeal     24       CONTRIBUTOR
    52                blink1073     21            MEMBER
    160                stonebig     18       CONTRIBUTOR
    77              drunkwcodes     18              NONE
    85             flying-sheep     17       CONTRIBUTOR
    142                 pdeitel     15              NONE
    42                     alok     15       CONTRIBUTOR
    46                  anntzer     12       CONTRIBUTOR
    78                  efiring     12       CONTRIBUTOR
    72                 dhomeier     12              NONE
    48                 asmeurer      9       CONTRIBUTOR
    53                  blueyed      9       CONTRIBUTOR
    82    faultdiagnosistoolbox      9              NONE
    154            sanketShetty      9              NONE
    14                    Jamim      9       CONTRIBUTOR
    28         RafalSkolasinski      9              NONE
    101              jamesmyatt      9              NONE
    137                  natsrm      9              NONE
    174                willingc      7            MEMBER
    47             arogozhnikov      6              NONE
    61                 cottrell      6       CONTRIBUTOR
    45           angertdevsingh      6              NONE
    172                voronind      6              NONE
    11            FabioLuporini      6              NONE
    134                 mlucool      6              NONE
    146              pierstitus      6       CONTRIBUTOR
    80                 ericdill      6              NONE
    83                 fehiepsi      6              NONE
    117                  kmaork      6       CONTRIBUTOR
    179               ztaylor96      6              NONE
    1    AbsurdMagpieScrutinies      6              NONE
    110                jseabold      6       CONTRIBUTOR
    108        jonathanslenders   

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 author  n_first_responses authorAssociation
    2            choldgraf                  4            MEMBER
    0  3keepmovingforward3                  1              NONE
    1              betatim                  1            MEMBER
    3          cornhundred                  1              NONE
    4      matthewfeickert                  1       CONTRIBUTOR
    5             stwunsch                  1              NONE
    6         taylorreiter                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 90 days',
    width: 1000
  }), Chart({
    data:                      author  n_first_responses authorAssociation
    4                   Carreau                 70            MEMBER
    83                    minrk                 10            MEMBER
    8            LucianaMarques                  6       CONTRIBUTOR
    57                 gpotter2                  5       CONTRIBUTOR
    94                 rmorshea                  4      COLLABORATOR
    51             flying-sheep                  3       CONTRIBUTOR
    36               ccordoba12                  2       CONTRIBUTOR
    43                 dhomeier                  2              NONE
    47              drunkwcodes                  2              NONE
    99                 stonebig                  2       CONTRIBUTOR
    60                  hroncok                  2       CONTRIBUTOR
    102               tacaswell                  2       CONTRIBUTOR
    64               jamesmyatt                  2              NONE
    7                     Jamim                  2       CONTRIBUTOR
    106                willingc                  2            MEMBER
    32                  blueyed                  2       CONTRIBUTOR
    78                   lidd77                  1              NONE
    77                   lheagy                  1              NONE
    76                  leogout                  1              NONE
    75                leifwalsh                  1              NONE
    74                  kmcminn                  1              NONE
    73                 khughitt                  1              NONE
    0    AbsurdMagpieScrutinies                  1              NONE
    72                    kernc                  1              NONE
    71                keflavich                  1              NONE
    70              justgfather                  1              NONE
    79                 m0nzderr                  1              NONE
    68           joaomarcosgris                  1              NONE
    67                 jfhbrook                  1              NONE
    66                   jdatin                  1              NONE
    65               jasongrout                  1            MEMBER
    63                  jakevdp                  1       CONTRIBUTOR
    62                 jaamarks                  1              NONE
    69       jorisvandenbossche                  1       CONTRIBUTOR
    84                  mjkvaak                  1              NONE
    80                   mag-al                  1              NONE
    81               matthewgdv                  1              NONE
    108           

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>